Done by [Sasa Buklijas](http://buklijas.info/blog/)

In [179]:
%%html
<style>
textarea
{
font-family: monospace;
//font-family: serif;
    
resize: none;
}

// source: https://stackoverflow.com/a/33570738/2006674
</style>

In [180]:
import ipywidgets as widgets
import time
import random

In [181]:
class GameState:
    def __init__(self):        
        self.car_index = None
        self.user_index = None
        #where_to_switch = None
    
    def door_select(self, door):
        choice = [1, 0, 0]
        random.shuffle(choice)
        self.user_index = door
        self.car_index = choice.index(1)

        # available indexes
        available_indexes = [0, 1, 2]
        available_indexes.remove(self.user_index)
        if self.car_index != self.user_index:
            available_indexes.remove(self.car_index)

        index_to_open = random.choice(available_indexes)

        where_to_switch = [0, 1, 2]
        where_to_switch.remove(self.user_index)
        where_to_switch.remove(index_to_open)
        
        return(self.user_index, index_to_open, where_to_switch)
        
    def game_won_if_stay(self):
        if self.user_index == self.car_index:
            return True
        else:
            return False

class Statistics:
    def __init__(self):        
        self.won_switched = 0
        self.won_stayed = 0
        self.lost_switched = 0
        self.lost_stayed = 0
        
    def show_result(self):
        games_played = self.won_switched + self.won_stayed + self.lost_switched + self.lost_stayed
        gpsw = self.won_switched + self.lost_switched
        gpst = self.won_stayed + self.lost_stayed
        
        if gpsw == 0:
            pwsw = 0
            plsw = 0
        else:
            pwsw = self.won_switched/gpsw*100
            plsw = self.lost_switched/gpsw*100            
            
        if gpst == 0:
            pwst = 0
            plst = 0
        else:   
            pwst = self.won_stayed/gpst*100
            plst = self.lost_stayed/gpst*100
        
        res = 'Games played: %d     \n' % (games_played)
        res+= '         |    WON   |   LOST   |  TOTAL\n'
        res+= '------------------------------------------\n'
        res+= 'switched | {:3} {:3.0f}% | {:3} {:3.0f}% | {:3} {:3.0f}%\n'.format(self.won_switched, pwsw, self.lost_switched, plsw, gpsw, pwsw+plsw)
        res+= 'stayed   | {:3} {:3.0f}% | {:3} {:3.0f}% | {:3} {:3.0f}%\n'.format(self.won_stayed, pwst, self.lost_stayed, plst, gpst, pwst+plst)
        res+= 'TOTAL    |          |          | {:3}'.format(gpsw+gpst)
        
        return res

        
gs = GameState()
stat = Statistics()

# methods
def disable_all_door_buttons(state):
    door1.disabled=state
    door2.disabled=state
    door3.disabled=state

def door1_selected(event):
    label2.value = 'You have selected Door 1'
    door1.button_style='warning' 
    start_game(0)

def door2_selected(event):
    label2.value = 'You have selected Door 2'
    door2.button_style='warning' 
    start_game(1)

def door3_selected(event):
    label2.value = 'You have selected Door 3'
    door3.button_style='warning' 
    start_game(2)

def start_game(selected_index):
    disable_all_door_buttons(True)
    time.sleep(0.5)
        
    user_index, index_to_open, where_to_switch = gs.door_select(selected_index)
    
    # open goat door
    label3.value += 'After Door %d there is GOTA' % (index_to_open+1)
    time.sleep(.5)
    
    if index_to_open == 0:
        door1.description = 'GOAT'
        door1.button_style = 'danger'
    elif index_to_open == 1:
        door2.description = 'GOAT'
        door2.button_style = 'danger'
    elif index_to_open == 2:
        door3.description = 'GOAT'
        door3.button_style = 'danger'
    time.sleep(0.5)
    
    label4.value += 'Do you want to switch from Door %d to Door %d ?' % (user_index+1, where_to_switch[0]+1)
    time.sleep(0.5)
    yes.layout.visibility = 'visible'
    time.sleep(0.25)
    no.layout.visibility = 'visible'
    time.sleep(0.5)

def yes_selected(event):  
    if report_result_of_game(not(gs.game_won_if_stay()), 'You switched, '):
        stat.won_switched += 1
    else:
        stat.lost_switched += 1
        
    label0.value = stat.show_result()
    
def no_selected(event):    
    if report_result_of_game(gs.game_won_if_stay(), 'You stayed, '):
        stat.won_stayed += 1
    else:
        stat.lost_stayed += 1  
        
    label0.value = stat.show_result()
    
def report_result_of_game(state, prefix):
    yes.disabled=True
    no.disabled=True
    
    result = prefix
    
    if state:
        result += 'Excellent choice, you have WON !'
    else:
        result += 'Bad choice, you have LOST.'
        
    label5.value = result
    
    # show replay
    time.sleep(0.5)
    replay.layout.visibility = 'visible'
    
    return state
    
def do_replay(event):
    replay.layout.visibility = 'hidden'
    yes.layout.visibility = 'hidden'
    no.layout.visibility = 'hidden'
    yes.disabled = False
    no.disabled = False
    label5.value = ''
    label4.value = ''
    label3.value = ''
    label2.value = ''
    door3.description='Door 3'
    door3.button_style = ''
    door2.description='Door 2'
    door2.button_style = ''
    door1.description='Door 1'
    door1.button_style = ''
    disable_all_door_buttons(False)
    

# UI
label0 = widgets.Textarea(disabled=True, layout={'height': '130px', 'width': '400px'})
label01 = widgets.Label()
label02 = widgets.Label()
label03 = widgets.Label()
label04 = widgets.Label()
label05 = widgets.Label()
label = widgets.Label(value='Select one door')
door1 = widgets.Button(description='Door 1', button_style='', tooltip='Door 1')
door2 = widgets.Button(description='Door 2', button_style='', tooltip='Door 2')
door3 = widgets.Button(description='Door 3', button_style='', tooltip='Door 3')
label2 = widgets.Label()
label3 = widgets.Label()
label4 = widgets.Label()

yes = widgets.Button(description='YES', button_style='info', tooltip='YES')
no = widgets.Button(description='NO', button_style='success', tooltip='NO')
yes.layout.visibility = 'hidden'
no.layout.visibility = 'hidden'

label5 = widgets.Label()

replay = widgets.Button(description='Replay Game', button_style='', tooltip='Replay Game')
replay.layout.visibility = 'hidden'

# Interactions
door1.on_click(door1_selected)
door2.on_click(door2_selected)
door3.on_click(door3_selected)
yes.on_click(yes_selected)
no.on_click(no_selected)
replay.on_click(do_replay)

# UI Layout
top_box = widgets.VBox([label0, label])
middle_box = widgets.HBox([door1, door2, door3])
down_box = widgets.VBox([label2, label3, widgets.HBox([label4, yes, no]), label5])
down_box1 = widgets.VBox([replay])
widgets.VBox([top_box, middle_box, down_box, down_box1])